# Equilibrate extension
Notebook to illustrate the calculation of forcing a silicate liquid to be in equilibrium with both quartz, plagioclase and sanidine at a specifed temperature and pressure

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as opt
import scipy.linalg as lin 
%matplotlib inline

### Get class instances to calculate the properties of quartz from Berman (1988)

In [ ]:
from thermoengine import phases

In [ ]:
thermoDB = phases.ThermoDB()
quartz = thermoDB.new_phase('Qz')

### Get class instances to calculate the properties of silicate liquid and feldspar from MELTS 1.0.2

In [ ]:
import ctypes
from ctypes import cdll
from ctypes import util
from rubicon.objc import ObjCClass, objc_method
cdll.LoadLibrary(util.find_library('phaseobjc'))

In [ ]:
LiquidMelts = ObjCClass('LiquidMelts')
liquid = LiquidMelts.alloc().init()
FeldsparBerman = ObjCClass('FeldsparBerman')
plagioclase = FeldsparBerman.alloc().initWithCompositionConstraint_('Plagioclase')
sanidine = FeldsparBerman.alloc().initWithCompositionConstraint_('Sanidine')

### Set number of components in the system and the number of fixed phases

In [ ]:
c = 5
f = 7

### The amount of quartz in the system is arbitrary ...
so, we fix its amount at a constant value

In [ ]:
nPhaseFix = 1.0

### Generate some information about the feldspar solid solution 
Set initial composition of plagioclase and sanidine just for testing purposes

In [ ]:
ncFeldspar = plagioclase.numberOfSolutionComponents()
print ('Feldspar: number of solution components = ', ncFeldspar)
mPlag = (ctypes.c_double*ncFeldspar)()
ctypes.cast(mPlag, ctypes.POINTER(ctypes.c_double))
mAlk = (ctypes.c_double*ncFeldspar)()
ctypes.cast(mAlk, ctypes.POINTER(ctypes.c_double))
mPlag[0] = 0.8
mPlag[1] = 0.1
mPlag[2] = 0.1
mAlk[0] = 0.3
mAlk[1] = 0.1
mAlk[2] = 0.6
for i in range(0,ncFeldspar):
    print ('Component No: {0:1d} is {1:<15.15s} Initial guess for Plag = {2:5.2f} Alk = {3:5.2f}'.format( \
    i, plagioclase.nameOfSolutionSpeciesAtIndex_(i), mPlag[i], mAlk[i]))

### Choose an initial bulk composition for the silicate liquid 
The initial composition listed below is a high-silica rhyolite projected into the system SiO<sub>2</sub>-Al<sub>2</sub>O<sub>3</sub>-CaO-Na<sub>2</sub>O-K<sub>2</sub>O

In [ ]:
nc = liquid.numberOfSolutionComponents()
bc = np.zeros((c,1))
def setBulkComposition(nSiO2=0.665792, nAl2O3=0.042436, nCaSiO3=0.004596, nNa2SiO3=0.038493, nKAlSiO4=0.062105, \
                       nQz=nPhaseFix, nPlagAb=0.8, nPlagAn=0.1, nPlagSn=0.1, nAlkAb=0.3, nAlkAn=0.1, nAlkSn=0.6):
    bc[0] = nSiO2 + nCaSiO3 + nNa2SiO3 + nKAlSiO4 + nQz + 3.0*nPlagAb + 2.0*nPlagAn + 3.0*nPlagSn \
          + 3.0*nAlkAb + 2.0*nAlkAn + 3.0*nAlkSn
    bc[1] = nAl2O3 + nKAlSiO4/2.0 + nPlagAb/2.0 + nPlagAn + nPlagSn/2.0 + nAlkAb/2.0 + nAlkAn + nAlkSn/2.0
    bc[2] = nCaSiO3 + nPlagAn + nAlkAn
    bc[3] = nNa2SiO3 + nPlagAb/2.0 + nAlkAb/2.0
    bc[4] = nKAlSiO4/2.0 + nPlagSn/2.0 + nAlkSn/2.0
m = (ctypes.c_double*nc)()
ctypes.cast(m, ctypes.POINTER(ctypes.c_double))
def setMoles(nSiO2=0.665792, nAl2O3=0.042436, nCaSiO3=0.004596, nNa2SiO3=0.038493, nKAlSiO4=0.062105, \
            nPlagAb=0.8, nPlagAn=0.1, nPlagSn=0.1, nAlkAb=0.3, nAlkAn=0.1, nAlkSn=0.6):
    nLiq = np.zeros((c,1))
    nLiq[0] = nSiO2
    nLiq[1] = nAl2O3
    nLiq[2] = nCaSiO3
    nLiq[3] = nNa2SiO3
    nLiq[4] = nKAlSiO4
    m[0] = nLiq[0]  # SiO2
    m[1] = 0.0      # TiO2
    m[2] = nLiq[1]  # Al2O3
    m[3] = 0.0      # Fe2O3
    m[4] = 0.0      # Cr2O3
    m[5] = 0.0      # Fe2SiO4
    m[6] = 0.0      # Mn2SiO4
    m[7] = 0.0      # Mg2SiO4
    m[8] = 0.0      # NiSi1/2O2
    m[9] = 0.0      # CoSi1/2O2
    m[10] = nLiq[2] # CaSiO3
    m[11] = nLiq[3] # Na2SiO3
    m[12] = nLiq[4] # KAlSiO4
    m[13] = 0.0     # Ca3(PO4)2
    m[14] = 0.0     # H2O
    mPlag[0] = nPlagAb
    mPlag[1] = nPlagAn
    mPlag[2] = nPlagSn
    mAlk[0]  = nAlkAb
    mAlk[1]  = nAlkAn
    mAlk[2]  = nAlkSn
    return nLiq

### Choose a temperature and pressure
The thermodynamic properties of quartz are only functions of temperature and pressure

In [ ]:
t = 755.0 + 273.15 # K
p = 1750.0 # bars
mu0Qz = quartz.get_gibbs_energy(t, p)[0]

### Find an initial guess that is consistent with the constraints
Use the scipy optimization function minimize to solve the non-linear system of chemical potential equalities subject to bound constraints

In [ ]:
def fcon(x):
    setMoles(nSiO2=x[0], nAl2O3=x[1], nCaSiO3=x[2], nNa2SiO3=x[3], nKAlSiO4=x[4], \
            nPlagAb=x[5], nPlagAn=x[6], nPlagSn=x[7], nAlkAb=x[8], nAlkAn=x[9], nAlkSn=x[10])
    muLiq = liquid.getChemicalPotentialFromMolesOfComponents_andT_andP_(m, t, p)
    result = (mu0Qz-muLiq.valueAtIndex_(0))**2
    omegaAb = 5.0*muLiq.valueAtIndex_(0)/2.0 + muLiq.valueAtIndex_(2)/2.0 + muLiq.valueAtIndex_(11)/2.0
    omegaAn = muLiq.valueAtIndex_(0) + muLiq.valueAtIndex_(2) + muLiq.valueAtIndex_(10)
    omegaSn = 2.0*muLiq.valueAtIndex_(0) + muLiq.valueAtIndex_(12)
    muPlag = plagioclase.getChemicalPotentialFromMolesOfComponents_andT_andP_(mPlag, t, p)
    result += (muPlag.valueAtIndex_(0) - omegaAb)**2
    result += (muPlag.valueAtIndex_(1) - omegaAn)**2
    result += (muPlag.valueAtIndex_(2) - omegaSn)**2
    muAlk = sanidine.getChemicalPotentialFromMolesOfComponents_andT_andP_(mAlk, t, p)
    result += (muAlk.valueAtIndex_(0) - omegaAb)**2
    result += (muAlk.valueAtIndex_(1) - omegaAn)**2
    result += (muAlk.valueAtIndex_(2) - omegaSn)**2
    return result
result = opt.minimize(fcon,np.array([0.665792, 0.042436, 0.004596, 0.038493, 0.062105, 0.8, 0.1, 0.1, 0.3, 0.1, 0.6]), \
                      bounds=((0,None), (0,None), (0,None), (0,None), (0,None), (0,None), \
                              (0,None), (0,None), (0,None), (0,None), (0,None)))
print (result)
nLiqSiO2 = result.x[0]
nLiqAl2O3 = result.x[1]
nLiqCaSiO3 = result.x[2]
nLiqNa2SiO3 = result.x[3]
nLiqKAlSiO4 = result.x[4]
nPlagAb = result.x[5]
nPlagAn = result.x[6]
nPlagSn = result.x[7]
nAlkAb = result.x[8]
nAlkAn = result.x[9]
nAlkSn = result.x[10]
nQz = nPhaseFix
setBulkComposition(nSiO2=nLiqSiO2, nAl2O3=nLiqAl2O3, nCaSiO3=nLiqCaSiO3, nNa2SiO3=nLiqNa2SiO3, \
                   nKAlSiO4=nLiqKAlSiO4, nQz=nQz, nPlagAb=nPlagAb, nPlagAn=nPlagAn, nPlagSn=nPlagSn, \
                   nAlkAb=nAlkAb, nAlkAn=nAlkAn, nAlkSn=nAlkSn)
print ('{0:<10.10s} {1:10.8f} moles'.format('SiO2', bc[0][0]))
print ('{0:<10.10s} {1:10.8f} moles'.format('Al2O3', bc[1][0]))
print ('{0:<10.10s} {1:10.8f} moles'.format('CaO', bc[2][0]))
print ('{0:<10.10s} {1:10.8f} moles'.format('Na2O', bc[3][0]))
print ('{0:<10.10s} {1:10.8f} moles'.format('K2O', bc[4][0]))
print ('Residuals:')
muLiq = liquid.getChemicalPotentialFromMolesOfComponents_andT_andP_(m, t, p)
print('{0:<10.10s} {1:13.6e} J'.format('Qz mu', mu0Qz-muLiq.valueAtIndex_(0)))
omegaAb = 5.0*muLiq.valueAtIndex_(0)/2.0 + muLiq.valueAtIndex_(2)/2.0 + muLiq.valueAtIndex_(11)/2.0
omegaAn = muLiq.valueAtIndex_(0) + muLiq.valueAtIndex_(2) + muLiq.valueAtIndex_(10)
omegaSn = 2.0*muLiq.valueAtIndex_(0) + muLiq.valueAtIndex_(12)
muPlag = plagioclase.getChemicalPotentialFromMolesOfComponents_andT_andP_(mPlag, t, p)
print('{0:<10.10s} {1:13.6e} J'.format('Plag Ab mu', muPlag.valueAtIndex_(0) - omegaAb))
print('{0:<10.10s} {1:13.6e} J'.format('Plag An mu', muPlag.valueAtIndex_(1) - omegaAn))
print('{0:<10.10s} {1:13.6e} J'.format('Plag Sn mu', muPlag.valueAtIndex_(2) - omegaSn))
muAlk = sanidine.getChemicalPotentialFromMolesOfComponents_andT_andP_(mAlk, t, p)
print('{0:<10.10s} {1:13.6e} J'.format('Alk Ab mu', muAlk.valueAtIndex_(0) - omegaAb))
print('{0:<10.10s} {1:13.6e} J'.format('Alk An mu', muAlk.valueAtIndex_(1) - omegaAn))
print('{0:<10.10s} {1:13.6e} J'.format('Alk Sn mu', muAlk.valueAtIndex_(2) - omegaSn))

# Set up bulk composition constraint matrix - matrix is constant
$$
\left[ {\begin{array}{*{20}{c}}
{{b_{{\rm{Si}}{{\rm{O}}_2}}}}\\
{{b_{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}}}\\
{{b_{{\rm{CaO}}}}}\\
{{b_{{\rm{N}}{{\rm{a}}_2}{\rm{O}}}}}\\
{{b_{{{\rm{K}}_2}{\rm{O}}}}}
\end{array}} \right] = \left[ {\begin{array}{*{20}{c}}
1&0&1&1&1&1&3&2&3&3&2&3\\
0&1&0&0&{\frac{1}{2}}&0&{\frac{1}{2}}&1&{\frac{1}{2}}&{\frac{1}{2}}&1&{\frac{1}{2}}\\
0&0&1&0&0&0&0&1&0&0&1&0\\
0&0&0&1&0&0&{\frac{1}{2}}&0&0&{\frac{1}{2}}&0&0\\
0&0&0&0&{\frac{1}{2}}&0&0&0&{\frac{1}{2}}&0&0&{\frac{1}{2}}
\end{array}} \right]\left[ {\begin{array}{*{20}{c}}
{n_{{\rm{Si}}{{\rm{O}}_2}}^{liq}}\\
{n_{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}\\
{n_{{\rm{CaSi}}{{\rm{O}}_3}}^{liq}}\\
{n_{{\rm{N}}{{\rm{a}}_2}{\rm{Si}}{{\rm{O}}_3}}^{liq}}\\
{n_{{\rm{KAlSi}}{{\rm{O}}_4}}^{liq}}\\
{{n_{Qz}}}\\
{n_{Ab}^{plag}}\\
{n_{An}^{plag}}\\
{n_{Sn}^{plag}}\\
{n_{Ab}^{alk}}\\
{n_{An}^{alk}}\\
{n_{Sn}^{alk}}
\end{array}} \right]
$$

In [ ]:
C = np.array([[1,0,1,1,1,1,3,2,3,3,2,3],[0,1,0,0,0.5,0,0.5,1,0.5,0.5,1,0.5],[0,0,1,0,0,0,0,1,0,0,1,0],\
              [0,0,0,1,0,0,0.5,0,0,0.5,0,0],[0,0,0,0,0.5,0,0,0,0.5,0,0,0.5]])

### ...  and project it into the null space of fixed constraints - yields a constant matrix

In [ ]:
eps = np.finfo(float).eps
CTf = C[:,c:].transpose()
U,S,VT = np.linalg.svd(CTf)
rank = 0
for i in range(0,c):
    if S[i] > 10.0*eps:
        rank +=1
print ('CTf rank: ',rank)
VTff = VT[rank:,:]
Cproj = np.matmul(VTff, C)
for i in range(0,Cproj.shape[0]):
    for j in range(0,Cproj.shape[1]):
        if abs(Cproj[i][j]) < 10.0*eps:
            Cproj[i][j] = 0.0
print ('Cproj:')
print (Cproj)
bcproj = np.matmul(VTff, bc)
print ('bcproj')
print (bcproj)

### Define the Khorzhinski potential function
$$
\begin{array}{c}
L = {G^{liq}}\left( {n_{{\rm{Si}}{{\rm{O}}_2}}^{liq},n_{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq},n_{{\rm{CaSi}}{{\rm{O}}_3}}^{liq},n_{{\rm{N}}{{\rm{a}}_{\rm{2}}}{\rm{Si}}{{\rm{O}}_3}}^{liq},n_{{\rm{KAlSi}}{{\rm{O}}_4}}^{liq}} \right) + {n_{Qz}}\mu _{Qz}^o + {G^{plag}}\left( {n_{Ab}^{plag},n_{An}^{plag},n_{Sn}^{plag}} \right) + {G^{alk}}\left( {n_{Ab}^{alk},n_{An}^{alk},n_{Sn}^{alk}} \right)\\
 { - {n_{Qz}}\mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq} - \left( {n_{Ab}^{plag} + n_{Ab}^{alk}} \right)\Omega _{Ab}^{liq} - \left( {n_{An}^{plag} + n_{An}^{alk}} \right)\Omega _{An}^{liq} - \left( {n_{Sn}^{plag} + n_{Sn}^{alk}} \right)\Omega _{Sn}^{liq}}
\end{array}
$$
where
$$
\begin{array}{l}
\Omega _{Ab}^{liq} = \frac{5}{2}\mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq} + \frac{1}{2}\mu _{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq} + \frac{1}{2}\mu _{{\rm{N}}{{\rm{a}}_{\rm{2}}}{\rm{Si}}{{\rm{O}}_3}}^{liq}\\
\Omega _{An}^{liq} = \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq} + \mu _{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq} + \frac{1}{2}\mu _{{\rm{CaSi}}{{\rm{O}}_3}}^{liq}\\
\Omega _{Sn}^{liq} = 2\mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq} + \mu _{{\rm{KAlSi}}{{\rm{O}}_4}}^{liq}
\end{array}
$$

In [ ]:
def Khorzhinskii(nLiqSiO2, nLiqAl2O3, nLiqCaSiO3, nLiqNa2SiO3, nLiqKAlSiO4, \
                 nQz, nPlagAb, nPlagAn, nPlagSn, nAlkAb, nAlkAn, nAlkSn, t=1100.0, p=1750.0):
    setMoles(nLiqSiO2, nLiqAl2O3, nLiqCaSiO3, nLiqNa2SiO3, nLiqKAlSiO4, \
             nPlagAb, nPlagAn, nPlagSn, nAlkAb, nAlkAn, nAlkSn)
    Gliq = liquid.getGibbsFreeEnergyFromMolesOfComponents_andT_andP_(m,t,p)
    GPlag = plagioclase.getGibbsFreeEnergyFromMolesOfComponents_andT_andP_(mPlag,t,p)
    GAlk = sanidine.getGibbsFreeEnergyFromMolesOfComponents_andT_andP_(mAlk,t,p)
    muLiq = liquid.getChemicalPotentialFromMolesOfComponents_andT_andP_(m, t, p)
    omegaAb = 5.0*muLiq.valueAtIndex_(0)/2.0 + muLiq.valueAtIndex_(2)/2.0 + muLiq.valueAtIndex_(11)/2.0
    omegaAn = muLiq.valueAtIndex_(0) + muLiq.valueAtIndex_(2) + muLiq.valueAtIndex_(10)
    omegaSn = 2.0*muLiq.valueAtIndex_(0) + muLiq.valueAtIndex_(12)
    result = Gliq + nQz*(mu0Qz-muLiq.valueAtIndex_(0)) + GPlag + GAlk
    result += - (nPlagAb+nAlkAb)*omegaAb - (nPlagAn+nAlkAn)*omegaAn - (nPlagSn+nAlkSn)*omegaSn
    return result

### Define the gradient of the Khorzhinskii potential
$$
{\bf{g}} = \left[ {\begin{array}{*{20}{c}}
{\mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq} - {n_{Qz}}\frac{{\partial \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial n_{{\rm{Si}}{{\rm{O}}_2}}^{liq}}} - \left( {n_{Ab}^{plag} + n_{Ab}^{alk}} \right)\frac{{\partial \Omega _{Ab}^{liq}}}{{\partial n_{{\rm{Si}}{{\rm{O}}_2}}^{liq}}} - \left( {n_{An}^{plag} + n_{An}^{alk}} \right)\frac{{\partial \Omega _{An}^{liq}}}{{\partial n_{{\rm{Si}}{{\rm{O}}_2}}^{liq}}} - \left( {n_{Sn}^{plag} + n_{Sn}^{alk}} \right)\frac{{\partial \Omega _{Sn}^{liq}}}{{\partial n_{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}}\\
{\mu _{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq} - {n_{Qz}}\frac{{\partial \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial n_{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}} - \left( {n_{Ab}^{plag} + n_{Ab}^{alk}} \right)\frac{{\partial \Omega _{Ab}^{liq}}}{{\partial n_{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}} - \left( {n_{An}^{plag} + n_{An}^{alk}} \right)\frac{{\partial \Omega _{An}^{liq}}}{{\partial n_{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}} - \left( {n_{Sn}^{plag} + n_{Sn}^{alk}} \right)\frac{{\partial \Omega _{Sn}^{liq}}}{{\partial n_{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}}}\\
{\mu _{{\rm{CaSi}}{{\rm{O}}_3}}^{liq} - {n_{Qz}}\frac{{\partial \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial n_{{\rm{CaSi}}{{\rm{O}}_3}}^{liq}}} - \left( {n_{Ab}^{plag} + n_{Ab}^{alk}} \right)\frac{{\partial \Omega _{Ab}^{liq}}}{{\partial n_{{\rm{CaSi}}{{\rm{O}}_3}}^{liq}}} - \left( {n_{An}^{plag} + n_{An}^{alk}} \right)\frac{{\partial \Omega _{An}^{liq}}}{{\partial n_{{\rm{CaSi}}{{\rm{O}}_3}}^{liq}}} - \left( {n_{Sn}^{plag} + n_{Sn}^{alk}} \right)\frac{{\partial \Omega _{Sn}^{liq}}}{{\partial n_{{\rm{CaSi}}{{\rm{O}}_3}}^{liq}}}}\\
{\mu _{{\rm{N}}{{\rm{a}}_2}{\rm{Si}}{{\rm{O}}_3}}^{liq} - {n_{Qz}}\frac{{\partial \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial n_{{\rm{N}}{{\rm{a}}_2}{\rm{Si}}{{\rm{O}}_3}}^{liq}}} - \left( {n_{Ab}^{plag} + n_{Ab}^{alk}} \right)\frac{{\partial \Omega _{Ab}^{liq}}}{{\partial n_{{\rm{N}}{{\rm{a}}_2}{\rm{Si}}{{\rm{O}}_3}}^{liq}}} - \left( {n_{An}^{plag} + n_{An}^{alk}} \right)\frac{{\partial \Omega _{An}^{liq}}}{{\partial n_{{\rm{N}}{{\rm{a}}_2}{\rm{Si}}{{\rm{O}}_3}}^{liq}}} - \left( {n_{Sn}^{plag} + n_{Sn}^{alk}} \right)\frac{{\partial \Omega _{Sn}^{liq}}}{{\partial n_{{\rm{N}}{{\rm{a}}_2}{\rm{Si}}{{\rm{O}}_3}}^{liq}}}}\\
{\mu _{{\rm{KAlSi}}{{\rm{O}}_4}}^{liq} - {n_{Qz}}\frac{{\partial \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial n_{{\rm{KAlSi}}{{\rm{O}}_4}}^{liq}}} - \left( {n_{Ab}^{plag} + n_{Ab}^{alk}} \right)\frac{{\partial \Omega _{Ab}^{liq}}}{{\partial n_{{\rm{KAlSi}}{{\rm{O}}_4}}^{liq}}} - \left( {n_{An}^{plag} + n_{An}^{alk}} \right)\frac{{\partial \Omega _{An}^{liq}}}{{\partial n_{{\rm{KAlSi}}{{\rm{O}}_4}}^{liq}}} - \left( {n_{Sn}^{plag} + n_{Sn}^{alk}} \right)\frac{{\partial \Omega _{Sn}^{liq}}}{{\partial n_{{\rm{KAlSi}}{{\rm{O}}_4}}^{liq}}}}\\
{\mu _{Qz}^o - \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}\\
{\mu _{Ab}^{plag} - \Omega _{Ab}^{liq}}\\
{\mu _{An}^{plag} - \Omega _{An}^{liq}}\\
{\mu _{Sn}^{plag} - \Omega _{Sn}^{liq}}\\
{\mu _{Ab}^{alk} - \Omega _{Ab}^{liq}}\\
{\mu _{An}^{alk} - \Omega _{An}^{liq}}\\
{\mu _{Sn}^{alk} - \Omega _{Sn}^{liq}}
\end{array}} \right]
$$

In [ ]:
def Gradient(nLiqSiO2, nLiqAl2O3, nLiqCaSiO3, nLiqNa2SiO3, nLiqKAlSiO4, \
            nQz, nPlagAb, nPlagAn, nPlagSn, nAlkAb, nAlkAn, nAlkSn, t=1100.0, p=1750.0):
    setMoles(nLiqSiO2, nLiqAl2O3, nLiqCaSiO3, nLiqNa2SiO3, nLiqKAlSiO4, \
             nPlagAb, nPlagAn, nPlagSn, nAlkAb, nAlkAn, nAlkSn)
    dgdm = liquid.getDgDmFromMolesOfComponents_andT_andP_(m, t, p)
    muLiq = liquid.getChemicalPotentialFromMolesOfComponents_andT_andP_(m, t, p)
    omegaAb = 5.0*muLiq.valueAtIndex_(0)/2.0 + muLiq.valueAtIndex_(2)/2.0 + muLiq.valueAtIndex_(11)/2.0
    omegaAn = muLiq.valueAtIndex_(0) + muLiq.valueAtIndex_(2) + muLiq.valueAtIndex_(10)
    omegaSn = 2.0*muLiq.valueAtIndex_(0) + muLiq.valueAtIndex_(12)
    d2gdm2 = liquid.getD2gDm2FromMolesOfComponents_andT_andP_(m, t, p)
    
    DomegaAb = lambda i: 5.0*d2gdm2.valueAtRowIndex_andColIndex_(0, i)/2.0 \
                           + d2gdm2.valueAtRowIndex_andColIndex_(2, i)/2.0 \
                           + d2gdm2.valueAtRowIndex_andColIndex_(11, i)/2.0
    DomegaAn = lambda i: d2gdm2.valueAtRowIndex_andColIndex_(0, i) \
                       + d2gdm2.valueAtRowIndex_andColIndex_(2, i) \
                       + d2gdm2.valueAtRowIndex_andColIndex_(10, i)
    DomegaSn = lambda i: 2.0*d2gdm2.valueAtRowIndex_andColIndex_(0, i) + d2gdm2.valueAtRowIndex_andColIndex_(12, i)
    
    muPlag = plagioclase.getChemicalPotentialFromMolesOfComponents_andT_andP_(mPlag, t, p)
    muAlk = sanidine.getChemicalPotentialFromMolesOfComponents_andT_andP_(mAlk, t, p)

    result = np.zeros((c+f,1))
    result[0] = dgdm.valueAtIndex_(0) - nQz*d2gdm2.valueAtRowIndex_andColIndex_(0, 0) \
                                      - (nPlagAb+nAlkAb)*DomegaAb(0) \
                                      - (nPlagAn+nAlkAn)*DomegaAn(0) \
                                      - (nPlagSn+nAlkSn)*DomegaSn(0)
    result[1] = dgdm.valueAtIndex_(2) - nQz*d2gdm2.valueAtRowIndex_andColIndex_(0, 2) \
                                      - (nPlagAb+nAlkAb)*DomegaAb(2) \
                                      - (nPlagAn+nAlkAn)*DomegaAn(2) \
                                      - (nPlagSn+nAlkSn)*DomegaSn(2)
    result[2] = dgdm.valueAtIndex_(10) - nQz*d2gdm2.valueAtRowIndex_andColIndex_(0, 10) \
                                       - (nPlagAb+nAlkAb)*DomegaAb(10) \
                                       - (nPlagAn+nAlkAn)*DomegaAn(10) \
                                       - (nPlagSn+nAlkSn)*DomegaSn(10)
    result[3] = dgdm.valueAtIndex_(11) - nQz*d2gdm2.valueAtRowIndex_andColIndex_(0, 11) \
                                       - (nPlagAb+nAlkAb)*DomegaAb(11) \
                                       - (nPlagAn+nAlkAn)*DomegaAn(11) \
                                       - (nPlagSn+nAlkSn)*DomegaSn(11)
    result[4] = dgdm.valueAtIndex_(12) - nQz*d2gdm2.valueAtRowIndex_andColIndex_(0, 12) \
                                       - (nPlagAb+nAlkAb)*DomegaAb(12) \
                                       - (nPlagAn+nAlkAn)*DomegaAn(12) \
                                       - (nPlagSn+nAlkSn)*DomegaSn(12)
    result[5] = mu0Qz - dgdm.valueAtIndex_(0)
    result[6] = muPlag.valueAtIndex_(0) - omegaAb
    result[7] = muPlag.valueAtIndex_(1) - omegaAn
    result[8] = muPlag.valueAtIndex_(2) - omegaSn
    result[9] = muAlk.valueAtIndex_(0) - omegaAb
    result[10] = muAlk.valueAtIndex_(1) - omegaAn
    result[11] = muAlk.valueAtIndex_(2) - omegaSn
    return result

### Define a function to compute the "A" constraint matrix
$$
{\bf{A}} = \left[ {\begin{array}{*{20}{c}}
0&{ - \frac{1}{2}}&{\frac{1}{2}}&{\frac{1}{2}}&0&0&0&0&0&0&0&0\\
{ - \frac{{\partial \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial n_{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}}&{ - \frac{{\partial \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial n_{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial n_{{\rm{CaSi}}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial n_{{\rm{N}}{{\rm{a}}_2}{\rm{Si}}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial n_{{\rm{KAlSi}}{{\rm{O}}_4}}^{liq}}}}&0&0&0&0&0&0&0\\
{ - \frac{{\partial \Omega _{Ab}^{liq}}}{{\partial n_{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}}&{ - \frac{{\partial \Omega _{Ab}^{liq}}}{{\partial n_{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \Omega _{Ab}^{liq}}}{{\partial n_{{\rm{CaSi}}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \Omega _{Ab}^{liq}}}{{\partial n_{{\rm{N}}{{\rm{a}}_2}{\rm{Si}}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \Omega _{Ab}^{liq}}}{{\partial n_{{\rm{KAlSi}}{{\rm{O}}_4}}^{liq}}}}&0&{\frac{{\partial \mu _{Ab}^{plag}}}{{\partial n_{Ab}^{plag}}}}&{\frac{{\partial \mu _{Ab}^{plag}}}{{\partial n_{An}^{plag}}}}&{\frac{{\partial \mu _{Ab}^{plag}}}{{\partial n_{Sn}^{plag}}}}&0&0&0\\
{ - \frac{{\partial \Omega _{An}^{liq}}}{{\partial n_{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}}&{ - \frac{{\partial \Omega _{An}^{liq}}}{{\partial n_{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \Omega _{An}^{liq}}}{{\partial n_{{\rm{CaSi}}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \Omega _{An}^{liq}}}{{\partial n_{{\rm{N}}{{\rm{a}}_2}{\rm{Si}}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \Omega _{An}^{liq}}}{{\partial n_{{\rm{KAlSi}}{{\rm{O}}_4}}^{liq}}}}&0&{\frac{{\partial \mu _{An}^{plag}}}{{\partial n_{Ab}^{plag}}}}&{\frac{{\partial \mu _{An}^{plag}}}{{\partial n_{An}^{plag}}}}&{\frac{{\partial \mu _{An}^{plag}}}{{\partial n_{Sn}^{plag}}}}&0&0&0\\
{ - \frac{{\partial \Omega _{Sn}^{liq}}}{{\partial n_{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}}&{ - \frac{{\partial \Omega _{Sn}^{liq}}}{{\partial n_{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \Omega _{Sn}^{liq}}}{{\partial n_{{\rm{CaSi}}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \Omega _{Sn}^{liq}}}{{\partial n_{{\rm{N}}{{\rm{a}}_2}{\rm{Si}}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \Omega _{Sn}^{liq}}}{{\partial n_{{\rm{KAlSi}}{{\rm{O}}_4}}^{liq}}}}&0&{\frac{{\partial \mu _{Sn}^{plag}}}{{\partial n_{Ab}^{plag}}}}&{\frac{{\partial \mu _{Sn}^{plag}}}{{\partial n_{An}^{plag}}}}&{\frac{{\partial \mu _{Sn}^{plag}}}{{\partial n_{Sn}^{plag}}}}&0&0&0\\
{ - \frac{{\partial \Omega _{Ab}^{liq}}}{{\partial n_{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}}&{ - \frac{{\partial \Omega _{Ab}^{liq}}}{{\partial n_{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \Omega _{Ab}^{liq}}}{{\partial n_{{\rm{CaSi}}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \Omega _{Ab}^{liq}}}{{\partial n_{{\rm{N}}{{\rm{a}}_2}{\rm{Si}}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \Omega _{Ab}^{liq}}}{{\partial n_{{\rm{KAlSi}}{{\rm{O}}_4}}^{liq}}}}&0&0&0&0&{\frac{{\partial \mu _{Ab}^{alk}}}{{\partial n_{Ab}^{alk}}}}&{\frac{{\partial \mu _{Ab}^{alk}}}{{\partial n_{An}^{alk}}}}&{\frac{{\partial \mu _{Ab}^{alk}}}{{\partial n_{Sn}^{alk}}}}\\
{ - \frac{{\partial \Omega _{An}^{liq}}}{{\partial n_{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}}&{ - \frac{{\partial \Omega _{An}^{liq}}}{{\partial n_{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \Omega _{An}^{liq}}}{{\partial n_{{\rm{CaSi}}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \Omega _{An}^{liq}}}{{\partial n_{{\rm{N}}{{\rm{a}}_2}{\rm{Si}}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \Omega _{An}^{liq}}}{{\partial n_{{\rm{KAlSi}}{{\rm{O}}_4}}^{liq}}}}&0&0&0&0&{\frac{{\partial \mu _{An}^{alk}}}{{\partial n_{Ab}^{alk}}}}&{\frac{{\partial \mu _{An}^{alk}}}{{\partial n_{An}^{alk}}}}&{\frac{{\partial \mu _{An}^{alk}}}{{\partial n_{Sn}^{alk}}}}\\
0&0&0&0&0&1&0&0&0&0&0&0
\end{array}} \right]
$$

In [ ]:
def Amatrix(nLiqSiO2, nLiqAl2O3, nLiqCaSiO3, nLiqNa2SiO3, nLiqKAlSiO4, \
            nQz, nPlagAb, nPlagAn, nPlagSn, nAlkAb, nAlkAn, nAlkSn, t=1100.0, p=1750.0):
    setMoles(nLiqSiO2, nLiqAl2O3, nLiqCaSiO3, nLiqNa2SiO3, nLiqKAlSiO4, \
             nPlagAb, nPlagAn, nPlagSn, nAlkAb, nAlkAn, nAlkSn)
    d2gdm2 = liquid.getD2gDm2FromMolesOfComponents_andT_andP_(m, t, p)
    d2gdm2Plag = plagioclase.getD2gDm2FromMolesOfComponents_andT_andP_(mPlag, t, p)
    d2gdm2Alk = sanidine.getD2gDm2FromMolesOfComponents_andT_andP_(mAlk, t, p)
    
    DomegaAb = lambda i: 5.0*d2gdm2.valueAtRowIndex_andColIndex_(0, i)/2.0 \
                           + d2gdm2.valueAtRowIndex_andColIndex_(2, i)/2.0 \
                           + d2gdm2.valueAtRowIndex_andColIndex_(11, i)/2.0
    DomegaAn = lambda i: d2gdm2.valueAtRowIndex_andColIndex_(0, i) \
                       + d2gdm2.valueAtRowIndex_andColIndex_(2, i) \
                       + d2gdm2.valueAtRowIndex_andColIndex_(10, i)
    DomegaSn = lambda i: 2.0*d2gdm2.valueAtRowIndex_andColIndex_(0, i) + d2gdm2.valueAtRowIndex_andColIndex_(12, i)
    
    bottom = np.zeros((f,c+f))
    bottom[0][0] = -d2gdm2.valueAtRowIndex_andColIndex_(0, 0)
    bottom[0][1] = -d2gdm2.valueAtRowIndex_andColIndex_(0, 2)
    bottom[0][2] = -d2gdm2.valueAtRowIndex_andColIndex_(0, 10)
    bottom[0][3] = -d2gdm2.valueAtRowIndex_andColIndex_(0, 11)
    bottom[0][4] = -d2gdm2.valueAtRowIndex_andColIndex_(0, 12)
    
    bottom[1][0] = -DomegaAb(0)
    bottom[1][1] = -DomegaAb(2)
    bottom[1][2] = -DomegaAb(10)
    bottom[1][3] = -DomegaAb(11)
    bottom[1][4] = -DomegaAb(12)
    bottom[1][6] = d2gdm2Plag.valueAtRowIndex_andColIndex_(0, 0)
    bottom[1][7] = d2gdm2Plag.valueAtRowIndex_andColIndex_(0, 1)
    bottom[1][8] = d2gdm2Plag.valueAtRowIndex_andColIndex_(0, 2)
    
    bottom[2][0] = -DomegaAn(0)
    bottom[2][1] = -DomegaAn(2)
    bottom[2][2] = -DomegaAn(10)
    bottom[2][3] = -DomegaAn(11)
    bottom[2][4] = -DomegaAn(12)
    bottom[2][6] = d2gdm2Plag.valueAtRowIndex_andColIndex_(1, 0)
    bottom[2][7] = d2gdm2Plag.valueAtRowIndex_andColIndex_(1, 1)
    bottom[2][8] = d2gdm2Plag.valueAtRowIndex_andColIndex_(1, 2)
    
    bottom[3][0] = -DomegaSn(0)
    bottom[3][1] = -DomegaSn(2)
    bottom[3][2] = -DomegaSn(10)
    bottom[3][3] = -DomegaSn(11)
    bottom[3][4] = -DomegaSn(12)
    bottom[3][6] = d2gdm2Plag.valueAtRowIndex_andColIndex_(2, 0)
    bottom[3][7] = d2gdm2Plag.valueAtRowIndex_andColIndex_(2, 1)
    bottom[3][8] = d2gdm2Plag.valueAtRowIndex_andColIndex_(2, 2)
    
    bottom[4][0] = -DomegaAb(0)
    bottom[4][1] = -DomegaAb(2)
    bottom[4][2] = -DomegaAb(10)
    bottom[4][3] = -DomegaAb(11)
    bottom[4][4] = -DomegaAb(12)
    bottom[4][9] = d2gdm2Alk.valueAtRowIndex_andColIndex_(0, 0)
    bottom[4][10] = d2gdm2Alk.valueAtRowIndex_andColIndex_(0, 1)
    bottom[4][11] = d2gdm2Alk.valueAtRowIndex_andColIndex_(0, 2)
    
    bottom[5][0] = -DomegaAn(0)
    bottom[5][1] = -DomegaAn(2)
    bottom[5][2] = -DomegaAn(10)
    bottom[5][3] = -DomegaAn(11)
    bottom[5][4] = -DomegaAn(12)
    bottom[5][9] = d2gdm2Alk.valueAtRowIndex_andColIndex_(1, 0)
    bottom[5][10] = d2gdm2Alk.valueAtRowIndex_andColIndex_(1, 1)
    bottom[5][11] = d2gdm2Alk.valueAtRowIndex_andColIndex_(1, 2)
    
    bottom[6][5] = 1.0
    
    result = np.vstack((Cproj, bottom))
    return result

### Define the Lagrangian of the Khorzhimskii function
$$
\begin{array}{c}
\Lambda  = {G^{liq}}\left( {n_{{\rm{Si}}{{\rm{O}}_2}}^{liq},n_{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq},n_{{\rm{CaSi}}{{\rm{O}}_3}}^{liq},n_{{\rm{N}}{{\rm{a}}_{\rm{2}}}{\rm{Si}}{{\rm{O}}_3}}^{liq},n_{{\rm{KAlSi}}{{\rm{O}}_4}}^{liq}} \right) + {n_{Qz}}\mu _{Qz}^o + {G^{plag}}\left( {n_{Ab}^{plag},n_{An}^{plag},n_{Sn}^{plag}} \right) + {G^{alk}}\left( {n_{Ab}^{alk},n_{An}^{alk},n_{Sn}^{alk}} \right)\\
 - {n_{Qz}}\mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq} - \left( {n_{Ab}^{plag} + n_{Ab}^{alk}} \right)\Omega _{Ab}^{liq} - \left( {n_{An}^{plag} + n_{An}^{alk}} \right)\Omega _{An}^{liq} - \left( {n_{Sn}^{plag} + n_{Sn}^{alk}} \right)\Omega _{Sn}^{liq}\\
 - {\lambda _b}\left[ {\frac{1}{2}\left( {n_{{\rm{CaSi}}{{\rm{O}}_3}}^{liq} + n_{{\rm{N}}{{\rm{a}}_2}{\rm{Si}}{{\rm{O}}_3}}^{liq} - n_{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}^{liq}} \right) - \frac{1}{2}\left( {{b_{{\rm{CaO}}}} + {b_{{\rm{N}}{{\rm{a}}_2}{\rm{O}}}} + {b_{{{\rm{K}}_2}{\rm{O}}}} - {b_{{\rm{A}}{{\rm{l}}_2}{{\rm{O}}_3}}}} \right)} \right] - {\lambda _{Qz}}\left( {\mu _{{\rm{Si}}{{\rm{O}}_2}}^{qtz} - \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}} \right) - \lambda _{Ab}^{plag}\left( {\mu _{Ab}^{plag} - \Omega _{Ab}^{liq}} \right)\\
 - \lambda _{An}^{plag}\left( {\mu _{An}^{plag} - \Omega _{An}^{liq}} \right) - \lambda _{Sn}^{plag}\left( {\mu _{Sn}^{plag} - \Omega _{Sn}^{liq}} \right) - \lambda _{Ab}^{alk}\left( {\mu _{Ab}^{alk} - \Omega _{Ab}^{liq}} \right) - \lambda _{An}^{alk}\left( {\mu _{An}^{alk} - \Omega _{An}^{liq}} \right) - {\lambda _{{n_{Qz}}}}\left( {{n_{Qz}} - {n_{Fix}}} \right)
\end{array}
$$

In [ ]:
def Lagrangian(nLiqSiO2, nLiqAl2O3, nLiqCaSiO3, nLiqNa2SiO3, nLiqKAlSiO4, \
               nQz, nPlagAb, nPlagAn, nPlagSn, nAlkAb, nAlkAn, nAlkSn, t=1100.0, p=1750.0):
    nLiq = setMoles(nLiqSiO2, nLiqAl2O3, nLiqCaSiO3, nLiqNa2SiO3, nLiqKAlSiO4, \
                    nPlagAb, nPlagAn, nPlagSn, nAlkAb, nAlkAn, nAlkSn)
    
    result = Khorzhinskii(nLiqSiO2, nLiqAl2O3, nLiqCaSiO3, nLiqNa2SiO3, nLiqKAlSiO4, \
                          nQz, nPlagAb, nPlagAn, nPlagSn, nAlkAb, nAlkAn, nAlkSn, t, p)
    
    muLiq = liquid.getChemicalPotentialFromMolesOfComponents_andT_andP_(m, t, p)
    omegaAb = 5.0*muLiq.valueAtIndex_(0)/2.0 + muLiq.valueAtIndex_(2)/2.0 + muLiq.valueAtIndex_(11)/2.0
    omegaAn = muLiq.valueAtIndex_(0) + muLiq.valueAtIndex_(2) + muLiq.valueAtIndex_(10)
    omegaSn = 2.0*muLiq.valueAtIndex_(0) + muLiq.valueAtIndex_(12)
    
    muPlag = plagioclase.getChemicalPotentialFromMolesOfComponents_andT_andP_(mPlag, t, p)
    muAlk = sanidine.getChemicalPotentialFromMolesOfComponents_andT_andP_(mAlk, t, p)
    
    nFix = np.zeros((f,1))
    n = np.vstack((nLiq,nFix))
    # now add the Lagrange terms
    temp = np.matmul(Cproj, n)
    temp = np.subtract(temp, bcproj)
    result -= (np.dot(temp.transpose(), xLambda[0])[0][0])
    result -= xLambda[1][0]*(mu0Qz-muLiq.valueAtIndex_(0)) 
    result -= xLambda[2][0]*(muPlag.valueAtIndex_(0)-omegaAb)
    result -= xLambda[3][0]*(muPlag.valueAtIndex_(1)-omegaAn)
    result -= xLambda[4][0]*(muPlag.valueAtIndex_(2)-omegaSn)
    result -= xLambda[5][0]*(muAlk.valueAtIndex_(0)-omegaAb)
    result -= xLambda[6][0]*(muAlk.valueAtIndex_(1)-omegaAn)
    result -= xLambda[7][0]*(nQz-nPhaseFix)
    return result

### Define the Wronskian of the Lagrangian function
Elements of the Wronskian are given by
$$
\begin{array}{c}
\frac{{{\partial ^2}\Lambda }}{{\partial {n_i}\partial {n_j}}} = \frac{{{\partial ^2}{G^{liq}}}}{{\partial {n_i}\partial {n_j}}} + \frac{{{\partial ^2}{G^{plag}}}}{{\partial {n_i}\partial {n_j}}} + \frac{{{\partial ^2}{G^{alk}}}}{{\partial {n_i}\partial {n_j}}} - {\delta _{i,Qz}}\frac{{\partial \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial {n_j}}} - {\delta _{j,Qz}}\frac{{\partial \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial {n_i}}} - \left( {{n_{Qz}} - {\lambda _{Qz}}} \right)\frac{{{\partial ^2}\mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial {n_i}\partial {n_j}}}\\
 - \left( {{\delta _{i,plag - Ab}} + {\delta _{i,alk - Ab}}} \right)\frac{{\Omega _{Ab}^{liq}}}{{\partial {n_j}}} - \left( {{\delta _{j,plag - Ab}} + {\delta _{j,alk - Ab}}} \right)\frac{{\Omega _{Ab}^{liq}}}{{\partial {n_i}}} - \left( {n_{Ab}^{plag} + n_{Ab}^{alk}} \right)\frac{{{\partial ^2}\Omega _{Ab}^{liq}}}{{\partial {n_i}\partial {n_j}}}\\
 - \left( {{\delta _{i,plag - An}} + {\delta _{i,alk - An}}} \right)\frac{{\Omega _{An}^{liq}}}{{\partial {n_j}}} - \left( {{\delta _{j,plag - An}} + {\delta _{j,alk - An}}} \right)\frac{{\Omega _{An}^{liq}}}{{\partial {n_i}}} - \left( {n_{An}^{plag} + n_{An}^{alk}} \right)\frac{{{\partial ^2}\Omega _{An}^{liq}}}{{\partial {n_i}\partial {n_j}}}\\
 - \left( {{\delta _{i,plag - Sn}} + {\delta _{i,alk - Sn}}} \right)\frac{{\Omega _{Sn}^{liq}}}{{\partial {n_j}}} - \left( {{\delta _{j,plag - Sn}} + {\delta _{j,alk - Sn}}} \right)\frac{{\Omega _{Sn}^{liq}}}{{\partial {n_i}}} - \left( {n_{Sn}^{plag} + n_{Sn}^{alk}} \right)\frac{{{\partial ^2}\Omega _{Sn}^{liq}}}{{\partial {n_i}\partial {n_j}}}\\
 - \lambda _{Ab}^{plag}\left( {\frac{{{\partial ^2}\mu _{Ab}^{plag}}}{{\partial {n_i}\partial {n_j}}} - \frac{{{\partial ^2}\Omega _{Ab}^{liq}}}{{\partial {n_i}\partial {n_j}}}} \right) - \lambda _{An}^{plag}\left( {\frac{{{\partial ^2}\mu _{An}^{plag}}}{{\partial {n_i}\partial {n_j}}} - \frac{{{\partial ^2}\Omega _{An}^{liq}}}{{\partial {n_i}\partial {n_j}}}} \right) - \lambda _{Sn}^{plag}\left( {\frac{{{\partial ^2}\mu _{Sn}^{plag}}}{{\partial {n_i}\partial {n_j}}} - \frac{{{\partial ^2}\Omega _{Sn}^{liq}}}{{\partial {n_i}\partial {n_j}}}} \right)\\
 - \lambda _{Ab}^{alk}\left( {\frac{{{\partial ^2}\mu _{Ab}^{alk}}}{{\partial {n_i}\partial {n_j}}} - \frac{{{\partial ^2}\Omega _{Ab}^{liq}}}{{\partial {n_i}\partial {n_j}}}} \right) - \lambda _{An}^{alk}\left( {\frac{{{\partial ^2}\mu _{An}^{alk}}}{{\partial {n_i}\partial {n_j}}} - \frac{{{\partial ^2}\Omega _{An}^{liq}}}{{\partial {n_i}\partial {n_j}}}} \right)
\end{array}
$$

In [ ]:
def Wronskian(nLiqSiO2, nLiqAl2O3, nLiqCaSiO3, nLiqNa2SiO3, nLiqKAlSiO4, \
               nQz, nPlagAb, nPlagAn, nPlagSn, nAlkAb, nAlkAn, nAlkSn, t=1100.0, p=1750.0):
    nLiq = setMoles(nLiqSiO2, nLiqAl2O3, nLiqCaSiO3, nLiqNa2SiO3, nLiqKAlSiO4, \
                    nPlagAb, nPlagAn, nPlagSn, nAlkAb, nAlkAn, nAlkSn)
    w = np.zeros((c+f,c+f))
    ind = [0,2,10,11,12]
    
    d2gdm2 = liquid.getD2gDm2FromMolesOfComponents_andT_andP_(m, t, p)
    for i in range(0,5):
        for j in range(i,5):
            w[i][j] += d2gdm2.valueAtRowIndex_andColIndex_(ind[i], ind[j])
            w[j][i]  = w[i][j]
    d2gdm2Plag = plagioclase.getD2gDm2FromMolesOfComponents_andT_andP_(mPlag, t, p)
    for i in range(0,3):
        for j in range(i,3):
            w[i+6][j+6] += d2gdm2Plag.valueAtRowIndex_andColIndex_(i, j)
            w[j+6][i+6]  = w[i+6][j+6]
    d2gdm2Alk = sanidine.getD2gDm2FromMolesOfComponents_andT_andP_(mAlk, t, p)
    for i in range(0,3):
        for j in range(i,3):
            w[i+9][j+9] += d2gdm2Alk.valueAtRowIndex_andColIndex_(i, j)
            w[j+9][i+9]  = w[i+9][j+9]
    
    for i in range(0,5):
        w[5][i] -= d2gdm2.valueAtRowIndex_andColIndex_(0, ind[i])
        w[i][5]  = w[5][i]
    
    DomegaAb = lambda i: 5.0*d2gdm2.valueAtRowIndex_andColIndex_(0, i)/2.0 \
                           + d2gdm2.valueAtRowIndex_andColIndex_(2, i)/2.0 \
                           + d2gdm2.valueAtRowIndex_andColIndex_(11, i)/2.0
    DomegaAn = lambda i: d2gdm2.valueAtRowIndex_andColIndex_(0, i) \
                       + d2gdm2.valueAtRowIndex_andColIndex_(2, i) \
                       + d2gdm2.valueAtRowIndex_andColIndex_(10, i)
    DomegaSn = lambda i: 2.0*d2gdm2.valueAtRowIndex_andColIndex_(0, i) + d2gdm2.valueAtRowIndex_andColIndex_(12, i)
    
    for i in range(0,5):
        w[6][i]  += -DomegaAb(ind[i])
        w[i][6]   = w[6][i]
        w[9][i]  += -DomegaAb(ind[i])
        w[i][9]   = w[9][i]
        w[7][i]  += -DomegaAn(ind[i])
        w[i][7]   = w[7][i]
        w[10][i] += -DomegaAn(ind[i])
        w[i][10]  = w[10][i]
        w[8][i]  += -DomegaSn(ind[i])
        w[i][8]   = w[8][i]
        w[11][i] += -DomegaSn(ind[i])
        w[i][11]  = w[11][i]
   
    d3gdm3 = liquid.getD3gDm3FromMolesOfComponents_andT_andP_(m, t, p)
    D2omegaAb = lambda i, j : 5.0*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(0, i, j)/2.0 \
                                + d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(2, i, j)/2.0 \
                                + d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(11, i, j)/2.0
    D2omegaAn = lambda i, j : d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(0, i, j) \
                            + d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(2, i, j) \
                            + d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(10, i, j)
    D2omegaSn = lambda i, j : 2.0*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(0, i, j) \
                                + d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(12, i, j)
    d3gdm3Plag = plagioclase.getD3gDm3FromMolesOfComponents_andT_andP_(mPlag, t, p)
    d3gdm3Alk  = sanidine.getD3gDm3FromMolesOfComponents_andT_andP_(mAlk, t, p)
    
    for i in range(0,5):
        for j in range(i,5):
            w[i][j] -= (nQz-xLambda[1][0])*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(0, ind[i], ind[j])
            w[i][j] -= (nPlagAb+nAlkAb-xLambda[2][0]-xLambda[5][0])*D2omegaAb(ind[i],ind[j])
            w[i][j] -= (nPlagAn+nAlkAn-xLambda[3][0]-xLambda[6][0])*D2omegaAn(ind[i],ind[j])
            w[i][j] -= (nPlagSn+nAlkSn-xLambda[4][0])*D2omegaSn(ind[i],ind[j])
            w[j][i]  = w[i][j]
    
    for i in range(0,3):
        for j in range(i,3):
            w[i+6][j+6] -= xLambda[2][0]*d3gdm3Plag.valueAtFirstIndex_andSecondIndex_andThirdIndex_(0, i, j)
            w[i+6][j+6] -= xLambda[3][0]*d3gdm3Plag.valueAtFirstIndex_andSecondIndex_andThirdIndex_(1, i, j)
            w[i+6][j+6] -= xLambda[4][0]*d3gdm3Plag.valueAtFirstIndex_andSecondIndex_andThirdIndex_(2, i, j)
            w[j+6][i+6]  = w[i+6][j+6]
            w[i+9][j+9] -= xLambda[5][0]*d3gdm3Alk.valueAtFirstIndex_andSecondIndex_andThirdIndex_(0, i, j)
            w[i+9][j+9] -= xLambda[6][0]*d3gdm3Alk.valueAtFirstIndex_andSecondIndex_andThirdIndex_(1, i, j)
            w[j+9][i+9]  = w[i+9][j+9]
    
    return w

### Form the QR decomposition of A
Note that A must be square, so zero filled rows are added 
$
{\bf{A}} = {\bf{QR}} = \left[ {\begin{array}{*{20}{c}}
{{{\bf{Q}}_1}}&{{{\bf{Q}}_2}}
\end{array}} \right]\left[ {\begin{array}{*{20}{c}}
{{{\bf{R}}_1}}\\
{\bf{0}}
\end{array}} \right]
$
and 
${{\bf{Q}}_2}^T$ is the matrix ${\bf{Z}}$  

In [ ]:
A = Amatrix(nLiqSiO2, nLiqAl2O3, nLiqCaSiO3, nLiqNa2SiO3, nLiqKAlSiO4, \
            nQz, nPlagAb, nPlagAn, nPlagSn, nAlkAb, nAlkAn, nAlkSn, t, p)
fmt = '{:9.1e}'*12
print ("A matrix:", A.shape)
for i in range(0, A.shape[0]):
    print (fmt.format(A[i][0], A[i][1], A[i][2], A[i][3], A[i][4], A[i][5], A[i][6], A[i][7], A[i][8], \
                      A[i][9], A[i][10], A[i][11]))
# use
bottom = np.zeros((4,c+f))
A = np.vstack((A, bottom))
Q,R = lin.qr(A,mode='full')
# or
#R,Q = lin.rq(A,mode='full')
print ("Q matrix:", Q.shape)
for i in range(0,Q.shape[0]):
    print (fmt.format(Q[i][0], Q[i][1], Q[i][2], Q[i][3], Q[i][4], Q[i][5], Q[i][6], Q[i][7], Q[i][8], \
                      Q[i][9], Q[i][10], Q[i][11]))
print ("R matrix:", R.shape)
for i in range(0,R.shape[0]):
    print (fmt.format(R[i][0], R[i][1], R[i][2], R[i][3], R[i][4], R[i][5], R[i][6], R[i][7], R[i][8], \
                      R[i][9], R[i][10], R[i][11]))
Z = Q[f+1:,:].transpose()
print ("Z matrix", Z.shape)
fmt = '{:9.1e}'*4
for i in range(0,Z.shape[0]):
    print (fmt.format(Z[i][0], Z[i][1], Z[i][2], Z[i][3]))

### Form the vector of Lagrange multipliers
$$
{\bf{g}} = {{\bf{A}}^T}\left[ {\begin{array}{*{20}{c}}
{{\lambda _{{n_1}}}}\\
 \vdots \\
{{\lambda _{{n_{c - f}}}}}\\
{{\lambda _{{\phi _1}}}}\\
 \vdots \\
{{\lambda _{{\phi _f}}}}
\end{array}} \right]
$$

In [ ]:
print ("Gradient:")
g = Gradient(nLiqSiO2, nLiqAl2O3, nLiqCaSiO3, nLiqNa2SiO3, nLiqKAlSiO4, \
            nQz, nPlagAb, nPlagAn, nPlagSn, nAlkAb, nAlkAn, nAlkSn, t, p)
for i in range(0,g.shape[0]):
    print ("{0:10.3e}".format(g[i][0]))
print ("Lambda:")
xLambda = np.linalg.lstsq(A.transpose(),g)[0]
for i in range(0,xLambda.shape[0]):
    print ("{0:10.3e}".format(xLambda[i][0]))

### Compute the Wronskian of the Lagrangian

In [ ]:
W = Wronskian(nLiqSiO2, nLiqAl2O3, nLiqCaSiO3, nLiqNa2SiO3, nLiqKAlSiO4, \
            nQz, nPlagAb, nPlagAn, nPlagSn, nAlkAb, nAlkAn, nAlkSn, t, p)
fmt = '{:9.1e}'*12
for i in range(0,W.shape[0]):
    print (fmt.format(W[i][0], W[i][1], W[i][2], W[i][3], W[i][4], W[i][5], W[i][6], W[i][7], W[i][8], 
                      W[i][9], W[i][10], W[i][11]))

### Next project the gradient and the Wronskian
The search direction ($\Delta {\bf{n}}_{_\phi }^i$) is given by
$$
{{\bf{Z}}^T}{\bf{WZ}}\Delta {\bf{n}}_\phi ^i =  - {{\bf{Z}}^T}{\bf{g}}
$$
and ${\bf{n}}_\phi ^{i + 1} = \Delta {\bf{n}}_{_\phi }^i + {\bf{n}}_\phi ^i$ gives the quadratic approximation to the minimum, which must be adjusted to maintain the non-linear equality constraints
$$
{\bf{\tilde n}}_\phi ^{i + 1} = s\Delta {\bf{n}}_\phi ^i + {\bf{n}}_\phi ^i + \Delta {\bf{n}}_\phi ^{corr}
$$

In [ ]:
ZTg = np.matmul(Z.transpose(),g)
ZTWZ = np.matmul(Z.transpose(), np.matmul(W,Z))
print("Z^T g", ZTg.shape)
for i in range(0,ZTg.shape[0]):
    print ("{0:10.3e}".format(ZTg[i][0]))
print("Z^T W Z", ZTWZ.shape)
for i in range(0,ZTWZ.shape[0]):
    print ("{0:10.3e} {1:10.3e} {2:10.3e} {3:10.3e}".format(ZTWZ[i][0], ZTWZ[i][1], ZTWZ[i][2], ZTWZ[i][3]))

### ... and solve for the correction vector

In [ ]:
x = lin.solve(ZTWZ, ZTg)
print ("Solution in the null space:")
for i in range(0,x.shape[0]):
    print ("{0:10.3e}".format(x[i][0]))
deltan = np.matmul(Z,x)
print ("Inflated Solution:")
for i in range(0,deltan.shape[0]):
    print ("{0:10.3e}".format(deltan[i][0]))

## Final solution

In [ ]:
print ("n SiO2    liq = {0:8.6f} -> {1:8.6f}".format(nLiqSiO2, nLiqSiO2+deltan[0][0]))
print ("n Al2O3   liq = {0:8.6f} -> {1:8.6f}".format(nLiqAl2O3, nLiqAl2O3+deltan[1][0]))
print ("n CaSiO3  liq = {0:8.6f} -> {1:8.6f}".format(nLiqCaSiO3, nLiqCaSiO3+deltan[2][0]))
print ("n Na2SiO3 liq = {0:8.6f} -> {1:8.6f}".format(nLiqNa2SiO3, nLiqNa2SiO3+deltan[3][0]))
print ("n KAlSiO4 liq = {0:8.6f} -> {1:8.6f}".format(nLiqKAlSiO4, nLiqKAlSiO4+deltan[4][0]))
print ("n Qz          = {0:8.6f} -> {1:8.6f}".format(nQz, nQz+deltan[5][0]))
print ("n Plag Ab     = {0:8.6f} -> {1:8.6f}".format(nPlagAb, nPlagAb+deltan[6][0]))
print ("n Plag An     = {0:8.6f} -> {1:8.6f}".format(nPlagAn, nPlagAn+deltan[7][0]))
print ("n Plag Sn     = {0:8.6f} -> {1:8.6f}".format(nPlagSn, nPlagSn+deltan[8][0]))
print ("n Alk  Ab     = {0:8.6f} -> {1:8.6f}".format(nAlkAb, nAlkAb+deltan[9][0]))
print ("n Alk  An     = {0:8.6f} -> {1:8.6f}".format(nAlkAn, nAlkAn+deltan[10][0]))
print ("n Alk  Sn     = {0:8.6f} -> {1:8.6f}".format(nAlkSn, nAlkSn+deltan[11][0]))